# Importance Sampling
Using a change of distribution, the variance of estimates can be substantially reduced.

In [ ]:
using Printf #string formatting
using Random # random number utilities
using Plots # plotting
using Statistics # statistical tools
using Distributions # useful distributions
using QuadGK
using HypothesisTests

In [ ]:
# for readability, these are good settings to use
default(xtickfontsize=14,  ytickfontsize=14, ztickfontsize=14, 
    guidefontsize=14, legendfontsize=12, lw=2,ms=8)

# Example
Estimate
$$
\int_0^1 \cos(x/5) e^{-5x}dx=\mathbb{E}[\cos(U/5)e^{-5U}], \quad U \sim U(0,1)
$$
by Monte Carlo

In [ ]:
g = x-> cos(x/5) * exp(-5*x)

In [ ]:
xx = 0:0.01:1
plot(xx, g.(xx), label="cos(x/5)exp(-5x)", lw=2)
xlabel!("x")

## Naive Estimate
Use $U(0,1)$ r.v.'s.

In [ ]:
n_samples = 10^2;

Random.seed!(100)
U_samples = rand(n_samples);
g_samples = g.(U_samples);

Eg_est = mean(g_samples)
@printf("Mean = %g\n", Eg_est)
@printf("Variance = %g\n", var(g_samples));
α = 0.05;
z=quantile(Normal(), 1-α/2);

Ztest = OneSampleZTest(mean(g_samples), std(g_samples), n_samples);
@printf("95%% CI = (%g, %g)", 
    Ztest.xbar-z*Ztest.stderr, 
    Ztest.xbar+z*Ztest.stderr);

## Importance Sampling

Use distribution with pdf
$$
f_Y(y) = \frac{e^{-5y}1_{[0,1]}(y)}{c}
$$
to write
$$
\mathbb{E}[\cos(X/5)e^{-5X}] = c\mathbb{E}[\cos(Y/5)].
$$
The normalization is 
$$
c = (1-e^{-5})/5.
$$

Observe that the $\cos(x/5)$ is a very slowly varying function over $[0,1]$.

In [ ]:
xx = 0:0.01:1
plot(xx, cos.(xx/5), label="cos(x/5)", lw=2)
xlabel!("x")

In [ ]:
# sample Y by inverse funciton sampling
function randY()
    return -0.2 * log(1- rand()*(1-exp(-5)))
end

In [ ]:
n_samples = 10^2

Random.seed!(100)
Y_samples = zeros(n_samples);
@. Y_samples = randY();

c = (0.2 * (1-exp(-5)));

g_samples = cos.(Y_samples/5)* c;
Eg_est = mean(g_samples)

@printf("Mean = %g\n", Eg_est)
@printf("Variance = %g\n", var(g_samples));
α = 0.05;
z=quantile(Normal(), 1-α/2);

Ztest = OneSampleZTest(mean(g_samples), std(g_samples), n_samples);
@printf("95%% CI = (%g, %g)", 
    Ztest.xbar-z*Ztest.stderr, 
    Ztest.xbar+z*Ztest.stderr);

In [ ]:
O = x-> exp(-sqrt(1-x^2))/sqrt(x)
xx = 0:0.001:1
plot(xx, O.(xx), label="Observable", lw=2)
xlabel!("x")

## Naive Estimate

In [ ]:
n_samples = 10^4;

Random.seed!(100)
U_samples = rand(n_samples);

O_samples = O.(U_samples);
O_est = mean(O_samples)
println("Mean = ", O_est)
println("Variance = ", var(O_samples))
Ztest = OneSampleZTest(mean(O_samples), std(O_samples), n_samples);
println(@sprintf("95%% CI = (%g, %g)", 
        Ztest.xbar-1.96*Ztest.stderr, 
        Ztest.xbar+1.96*Ztest.stderr))

## Importance Sampling
Use distribution with pdf
$$
f_Y(y) = \frac{1_{[0,1]}}{2 \sqrt{y}}
$$
to write
$$
\mathbb{E}[e^{-\sqrt{1-X^2}}/\sqrt{X}] = \mathbb{E}[e^{-\sqrt{1-Y^2}}]\mathbb{E}[1/\sqrt{X}].
$$
In this
$$
c = \mathbb{E}[1/\sqrt{X}] = 2.
$$

In this case, $Y\sim X^2$, with $X\sim U(0,1)$.

In [ ]:
n_samples = 10^4;

Random.seed!(100)
Y_samples = rand(n_samples).^2;

O_samples = 2*exp.(-sqrt.(1 .- Y_samples.^2));
O_est = mean(O_samples)
println("Mean = ", O_est)
println("Variance = ", var(O_samples))
Ztest = OneSampleZTest(mean(O_samples), std(O_samples), n_samples);
println(@sprintf("95%% CI = (%g, %g)", 
        Ztest.xbar-1.96*Ztest.stderr, 
        Ztest.xbar+1.96*Ztest.stderr))

In [ ]:
exact_answer = quadgk(O,0,1)[1];
println("Exact = ", exact_answer)